Collaborative filtering
-----------------------

Today we are going to lay the groundwork for a cluster of computational tools that go under the names "statistical modeling", "machine learning", and even "Artifical Intelligence". Our test case will be a simple **recommender system**. Last time we computed in earnest, Suman had provided you with some data about news articles and provided you with the basics of a **content-based** recommendation system. In this case, news items could be surfaced if you express interest in one topic or another. Other information about the stories could also be used to produce a ranking -- Suman provided details including the number of "likes" or tweets, the publisher, the publication date and so on. 

This is the first of three approaches Alex Spangler wrote about in his NYT piece on their [next generation news recommender](https://open.blogs.nytimes.com/2015/08/11/building-the-next-new-york-times-recommendation-engine/). The second approach can be implemented without any information about content, and instead relies on how users rate or access content. On Amazon or Yelp or TripAdvisor, we have ratings that people provide for services. At the Times, we don't really have ratings, but instead we have whether or not someone read an article. **Collaborative filtering** uses patterns in these activities to make recommendations. (We "filter" items, recommending just a few, and this is a "collaborative" act because it relies on the input of others.)

At the heart of collaborative filtering is something called a "utility matrix" or "incidence matrix". It's a table where rows refer to users (shoppers, readers), say, and columns are items (news articles, consumer goods). A recommendation system "fills in" or predicts how the user in row i will react to the object represented by column j -- that is, the table entry (i,j). Oh, math!

So, here's a cartoon. Let's decide if we are going to recommend a great article on Climate Change to Elise. Elise is represented by row i in the incidence matrix and our Climate Change article is represented by column j. In row i we have 0's and 1's -- a 0 if Elise has not read the article and a 1 if she has. Column j is also filled with 0's and 1's, but this time an entry is 0 if the corresponding reader hasn't selected the article and 1 if he has. Got it? A row refers to the choices by a single user, and a column refers to the choices made by all our users for a single article. 

Now, we have two possible ways to use our incidence matrix, our table of 0's and 1's, to predict whether Elise will like our Climate Change article. The first is to find readers that are "similar" to Elise in terms of reading habits. That is, they have read some of the same articles or have 0's and 1's in the same places across their rows. We can take some number of readers who are most similar to Elise (the number always being called k) and then see what fraction read the article in question. In the image below, the gray band refers to the k readers who are similar to Elise and the red stripe highlights a column representing their reading of the single article. 

<img src=http://compute-cuj.org/abcabc.001.jpeg width=700>

The second approach to fill in the (i,j) entry of a table like this would be to look at  articles similar to the Climate Change piece we want to recommend to Elise. Articles are similar if they were read (or not) by the same people. We can then take the k most similar articles and see what fraction of them Elise has read. If it's high, we recommend the article to her. The figure below is just the flip of the one we've seen. The grey band is now similar articles to the given one and the red stripe looks at whether or not Elise read these. 

<img src=http://compute-cuj.org/abcabc.002.jpeg width=700>

This seems pretty straightforward (I hope). The techniques here are referred to ask "k-nearest neighbors" (or KNN or kNN). It's actually a pretty powerful machine learning (well, statistical) technique. And we leverage this kind of scheme all the time -- your doctor, for example, distills "you" into a row in a table, a height, a weight, a blood pressure, maybe some facts from your medical history. She then gives you advice about dropping a few pounds, say by what the medical profession knows about people like you. The idea is that the health outcomes of people "similar" to you can help predict what's around the corner for you. 

The key ideas here are pretty fundamental and **they have to do with distance.** Rows, for example, can be compared. In the case of collaborative filtering, we can evaluate one reader relative to another, marking some as close (likeminded, similar tastes) and others as far (red state, different tastes). This is a basic, mathematical abstraction that machine learning applies to all tables. Rows are points in a "high-dimensional space". (The same is true for columns, of course, except that it's typical to have tables -- spreadsheets-- organized so that rows refer to units of observation and columns refer to their characteristics.)

Let's see an incidence matrix from a real application. This was given to us by Alex Spangler and his team at the New York Times. It represents the reading habits of 1,800 visitors to the times. For privacy, we've stripped off anything identifying. But, rows refer to users and columns to URLs. 

In [ ]:
from pandas import read_csv
incidence = read_csv("http://compute-cuj.org/incidence.csv")

In [ ]:
incidence.head(5)

The first five rows of the `incidence` data frame shows us whether reader 3, say, read URL1, URL2 and so on. What you notice is that there are a lot of zeroes. Most people don't read most of the New York Times. 

Let's have a look at how big this table is. 

In [ ]:
incidence.shape

So, 1,815 users and 10K or so URL's. These represent articles read by the users for a one-month period. (Which, again, I am not at liberty to divulge -- be happy you see a real example of one of these matrices!)

Row and column sums can be important. Summing down the columns counts how many people (out of 1,815) read accessed the URL. Here we `apply()` the `sum()` function down columns, indicated with `axis=0`. We sort the resuts and see that many pages have just a single reader. 

In [ ]:
incidence.apply(sum,axis=0).sort_values()

Here we compute just how many have only one reader. We take the column sum and ask for a boolean outcome, `True` if there was one reader and `False` for more. Adding these up turns `True` to 1 and `False` to zero. 

In [ ]:
sum( incidence.apply(sum,axis=0) == 1)

That means 4,221 or about 40% are read just by one person. How many have two? Three? Less than five?

In [ ]:
# your code here



Let's look at just the heavy users. These are people who have read more than, say, 25 articles. This requires an `apply()` of `sum()` this time along rows, or `axis=1`.

In [ ]:
heavy = incidence[incidence.apply(sum,axis=1) > 20]

In [ ]:
heavy.shape

How should we judge similarity between two readers? Again, we want them to have read much the same articles. A simple measure for that is the **Jaccard metric.** It is good for rows (or columns) that are made up of 0's and 1's. Essentially it looks at how many 1's the two readers have in common, divided by the total number of articles they accessed. Well, you subtract that radio from 1. So, if two readers have nothing in common, the metric will be 1, and if they have everything in common, it will be 0.

In [ ]:
# Jaccard distance

def dist(a,b):
    
    intersection = sum((a+b)==2)
    union = sum((a+b)>=1)
    
    # print intersection,union
    
    return 1.0-(0.0+intersection)/union

Let's try out the function. We'll make two series (Pandas' representation of a row or column) of 0's and 1's and computes their Jaccard distance. See if this makes sense. Change the 0's and 1's to all agree or disagree.

In [ ]:
from pandas import Series
x = Series([1,0,1,1])
y = Series([0,0,1,1])
dist(x,y)

Finally, to compute the distance between two users, we need to access rows. This is done using `.iloc[]` subsetting. We haven't seen this yet, but it gives us fine-grained control over the rows or columns we want to extract. The notation is 
>`row selection, column selection`

Before the comma refers to rows and after to columns. We can use single integers for a single row or column, a slice like `5:10` to get a range,  and the empty slice `:` to ask for all the rows or columns. The result is a Pandas Series. 

Here is user with ID 40. 

In [ ]:
heavy.iloc[40,:]

This person has read 114 articles in our month.

In [ ]:
sum(heavy.iloc[40,:])

And here is the distance between user 40 and user 10.

In [ ]:
dist(heavy.iloc[40,:],heavy.iloc[10,:])

This loop iterates through the entire set of rows and calculates the distance between user 40 and the rest. 

In [ ]:
for i in range(877):
    d = dist(heavy.iloc[20,:],heavy.iloc[i,:])
    print i,d

We are now going to build on this idea of distance, but use a different incidence matrix. It is perverse but gives you the idea... Now, let's take this notion of distance and push it really, um, far.